In [1]:
import os
import sys
import random
import json
import pandas as pd
import numpy as np
import geopandas as gpd

from h3 import h3
from keplergl import KeplerGl

sys.path.append('../src/')
from utils import save_pickle, load_pickle, make_h3_index

RES = 10
HEX_COLUMN = f'geo_h3_{RES}'

In [81]:
df_home_job['geometry']

0        POLYGON ((37.03351 55.14029, 37.03346 55.14478...
1        POLYGON ((37.04135 55.14032, 37.04130 55.14481...
2        POLYGON ((37.02562 55.14475, 37.02556 55.14924...
3        POLYGON ((37.03346 55.14478, 37.03341 55.14927...
4        POLYGON ((37.04130 55.14481, 37.04125 55.14930...
                               ...                        
10235    POLYGON ((37.17539 56.01212, 37.17535 56.01661...
10236    POLYGON ((37.18341 56.01214, 37.18337 56.01663...
10237    POLYGON ((37.19143 56.01216, 37.19139 56.01665...
10238    POLYGON ((37.19944 56.01218, 37.19941 56.01667...
10239    POLYGON ((37.18337 56.01663, 37.18334 56.02112...
Name: geometry, Length: 10240, dtype: geometry

In [2]:
df_home_job = pd.read_csv("01_CLocation_July.csv")
df_shape = gpd.read_file("fishnet2021.shp")
df_home_job.rename(columns={'zid': 'cell_zid'}, inplace=True)

df_home_job = pd.merge(df_home_job, df_shape, on = ['cell_zid'])
gdf_home_job = json.loads(gpd.GeoDataFrame(df_home_job).to_json())
df_home_job.head()

,cell_zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,geometry
0,32909,4,0,0,0,"POLYGON ((37.03351 55.14029, 37.03346 55.14478..."
1,32910,25,8,26,0,"POLYGON ((37.04135 55.14032, 37.04130 55.14481..."
2,33261,2,0,2,0,"POLYGON ((37.02562 55.14475, 37.02556 55.14924..."
3,33262,13,2,11,0,"POLYGON ((37.03346 55.14478, 37.03341 55.14927..."
4,33263,10,0,3,0,"POLYGON ((37.04130 55.14481, 37.04125 55.14930..."


In [70]:
gdf_home_job

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'cell_zid': 32909,
    'customers_cnt_day': 0,
    'customers_cnt_home': 4,
    'customers_cnt_job': 0,
    'customers_cnt_move': 0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[37.03351220866129, 55.14028664630594],
      [37.03345981389939, 55.144778403557694],
      [37.04130213212256, 55.14480816490281],
      [37.041353646223094, 55.14031640325421],
      [37.03351220866129, 55.14028664630594]]]}},
  {'id': '1',
   'type': 'Feature',
   'properties': {'cell_zid': 32910,
    'customers_cnt_day': 26,
    'customers_cnt_home': 25,
    'customers_cnt_job': 8,
    'customers_cnt_move': 0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[37.041353646223094, 55.14031640325421],
      [37.04130213212256, 55.14480816490281],
      [37.049144464726, 55.1448374222931],
      [37.04919509816609, 55.14034565578318],
      [37.041353646223094, 55.14031640325421]]]}},
  {'id': '2

In [37]:
df_shape

,cell_zid,geometry
0,0,"POLYGON ((38.64721 54.25524, 38.64733 54.25973..."
1,1,"POLYGON ((38.65488 54.25517, 38.65500 54.25966..."
2,2,"POLYGON ((38.66255 54.25510, 38.66268 54.25959..."
3,3,"POLYGON ((38.67022 54.25502, 38.67035 54.25951..."
4,4,"POLYGON ((38.67789 54.25495, 38.67802 54.25944..."
...,...,...
189887,190237,"POLYGON ((37.69324 56.95549, 37.69326 56.95998..."
189888,190238,"POLYGON ((37.70146 56.95548, 37.70148 56.95997..."
189889,190239,"POLYGON ((37.70968 56.95547, 37.70970 56.95996..."
189890,190240,"POLYGON ((37.71790 56.95545, 37.71792 56.95994..."


In [ ]:
#генерация рандомных точек для квадратного полигона с заданным количеством

In [3]:
def generate_points_in_bounds(poly, size):
    minx, miny, maxx, maxy = poly.bounds
    return [(random.uniform(minx, maxx), random.uniform(miny, maxy)) for i in range(size)]

In [4]:
list_points = []
for idx, row in df_home_job.iterrows():
    points = generate_points_in_bounds(row['geometry'], row['customers_cnt_home'])
    list_points.append(points)
list_points = [x for sublist in list_points for x in sublist]    

In [145]:
len(list_points)

8702403

In [ ]:
#подсчет населения в гексагонах
df_points = pd.DataFrame(list_points, columns = ['lon', 'lat'])
df_points[HEX_COLUMN] = make_h3_index(df_points, 'lat', 'lon', RES)
df_customers_cnt_home_hex = df_points.groupby(HEX_COLUMN, as_index=False)['lat'].count()
df_customers_cnt_home_hex.columns = [HEX_COLUMN, 'customers_cnt_home_hex']
df_customers_cnt_home_hex

In [167]:
df_customers_cnt_home_hex

,geo_h3_10,customers_cnt_home_hex
0,8a118180880ffff,1
1,8a1181808817fff,1
2,8a118180881ffff,1
3,8a118180882ffff,1
4,8a118180886ffff,2
...,...,...
162725,8a11aacddd97fff,6
162726,8a11aacddd9ffff,8
162727,8a11aacddda7fff,6
162728,8a11aacdddaffff,2


In [339]:
from tqdm import tqdm

In [340]:
def calculate_population_in_isochrones(df,column_with_isoch_polygon):
    '''
    пересечение полигонов с изохронами и нахождение количества человек в изохроне
    '''
    df_tmp = df[~df[column_with_isoch_polygon].isnull()]

    #df_tmp = df_tmp.rename(columns = {'geometry':'geometry_base','zid':'cell_zid'})

    df_tmp = df_tmp[['cell_zid',column_with_isoch_polygon]]
    df_tmp = df_tmp.rename(columns = {column_with_isoch_polygon:'geometry'})
    #df_tmp = df_tmp[['cell_zid','geometry']]
    df_tmp['geometry'] = gpd.GeoSeries.from_wkt(df_tmp['geometry'])
    gdf_iso = json.loads(gpd.GeoDataFrame(df_tmp).to_json())
    
    hexagon_cover_dict = {}
    for idx in tqdm(range(0, len(gdf_iso['features'])),total=len(gdf_iso)):
        #print(idx)
        id1 = gdf_iso['features'][idx]['properties']['cell_zid']
        coords = gdf_iso['features'][idx]['geometry']['coordinates']
        coords = [np.column_stack((np.array(coords[0])[:, 1],
                                                          np.array(coords[0])[:, 0])).tolist()]
        try:
            geoJson = {'type':'Polygon','coordinates': coords}
            hexagons = list(h3.polyfill(geoJson, 10))
            sum_by_hexagons = \
            df_customers_cnt_home_hex['customers_cnt_home_hex'][df_customers_cnt_home_hex.geo_h3_10.isin(hexagons)].sum()
        except:
            geoJson = {'type':'Polygon','coordinates': coords[0]}
            hexagons = list(h3.polyfill(geoJson, 10)) 
            sum_by_hexagons = \
            df_customers_cnt_home_hex['customers_cnt_home_hex'][df_customers_cnt_home_hex.geo_h3_10.isin(hexagons)].sum()
            #print(df_customers_cnt_home_hex['customers_cnt_home_hex'][df_customers_cnt_home_hex.geo_h3_10.isin(hexagons)].sum())
        #hexagon_cover_list.append(hexagons)
        hexagon_cover_dict[id1] = sum_by_hexagons
    return hexagon_cover_dict

In [341]:
#автомобильные изохроны

In [342]:
df_is = pd.read_csv("isochrones_all_new.csv",sep = ';')

In [343]:
hexagon_cover_dict_drive = calculate_population_in_isochrones(df = df_is[:10],column_with_isoch_polygon = 'pol_15min')

10it [00:00, 21.18it/s]                      


In [344]:
hexagon_cover_dict_drive

{32909: 0,
 32910: 0,
 33261: 0,
 33262: 4318,
 33263: 4337,
 33264: 4032,
 33265: 3968,
 33614: 3486,
 33615: 4310,
 33616: 4316}

In [345]:
list_zid = list(hexagon_cover_dict_drive.keys())
list_popul = list(hexagon_cover_dict_drive.values())
df_hex_drive = pd.DataFrame\
(list(zip(list_zid,list_popul)),columns = ['cell_zid','population'])
df_hex_drive.to_csv('population_in_15m_isochr_drive.csv',sep = ";")

In [346]:
#пешие изохроны

In [347]:
df_is_walk = pd.read_csv("izochrones_walk.csv",sep = ';')

In [348]:
df_is_walk = df_is_walk.rename(columns = {'geometry':'geometry_base','zid':'cell_zid'})

In [349]:
hexagon_cover_dict_walk = calculate_population_in_isochrones(df = df_is_walk[:10],column_with_isoch_polygon = 'pol_15min')

10it [00:00, 60.34it/s]              


In [350]:
hexagon_cover_dict_walk

{32909: 6,
 32910: 14,
 33261: 12,
 33262: 15,
 33263: 17,
 33264: 24,
 33265: 51,
 33614: 21,
 33615: 25,
 33616: 21}

In [351]:
list_zid = list(hexagon_cover_dict_walk.keys())
list_popul = list(hexagon_cover_dict_walk.values())
df_hex_drive = pd.DataFrame\
(list(zip(list_zid,list_popul)),columns = ['cell_zid','population'])
df_hex_drive.to_csv('population_in_15m_isochr_walk.csv',sep = ";")